In [2]:
%%pyspark

df = spark.read.load('abfss://capture@splacceler5lmevhdeon4ym.dfs.core.windows.net/SeattlePublicLibrary/Checkouts_By_Title__Physical_Items_.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

In [3]:
%%pyspark

# Show Schema
df.printSchema()

In [4]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
capture_account_name = 'splacceler5lmevhdeon4ym' # fill in your primary account name
capture_container_name = 'capture' # fill in your container name
capture_relative_path = 'SeattlePublicLibrary/Checkouts_By_Title__Physical_Items_.csv' # fill in your relative folder path

capture_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (capture_container_name, capture_account_name, capture_relative_path)
print('Primary storage account path: ' + capture_adls_path)

In [5]:
%%pyspark

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType
csvSchema = StructType([
  StructField('id', StringType(), True), 
  StructField('checkout_year', IntegerType(), True),
  StructField('bib_number', IntegerType(), True),  
  StructField('item_barcode', StringType(), True),
  StructField('item_type', StringType(), True),
  StructField('collection', StringType(), True),
  StructField('call_number', StringType(), True),
  StructField('item_title', StringType(), True),
  StructField('subjects', StringType(), True),
  StructField('CheckoutDateTime', StringType(), True)      
])

CheckByTPI_capture_df = spark.read.format('csv').option('header', 'True').schema(csvSchema).load(capture_adls_path)

display(CheckByTPI_capture_df.limit(10))



In [21]:
%%pyspark

from pyspark.sql.functions import to_date, to_timestamp, col, date_format, current_timestamp
df_final = (CheckByTPI_capture_df.withColumn("checkout_date_time", to_timestamp(col("CheckoutDateTime"),"mm/dd/yyyy hh:mm:ss aa")).drop("CheckoutDateTime")
                                 .withColumn('loadDate', date_format(current_timestamp(), 'MM/dd/yyyy hh:mm:ss aa'))
                                 .withColumn("load_date", to_timestamp(col("loadDate"),"MM/dd/yyyy hh:mm:ss aa")).drop("loadDate")
) 

In [22]:
%%pyspark

# Show Schema
df_final.printSchema()

display(df_final.limit(10))


In [9]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
compose_account_name = 'splacceler5lmevhdeon4ym' # fill in your primary account name
compose_container_name = 'compose' # fill in your container name
compose_relative_path = 'SeattlePublicLibrary/CheckoutsByTitlePhysicalItems/' # fill in your relative folder path

compose_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (compose_container_name, compose_account_name, compose_relative_path)
print('Primary storage account path: ' + compose_adls_path)

In [10]:
%%pyspark

compose_parquet_path = compose_adls_path + 'checkouts.parquet'

print('parquet file path: ' + compose_parquet_path)

In [11]:
%%pyspark

df_final.write.parquet(compose_parquet_path, mode = 'overwrite')

In [48]:
%%sql

-- Create database SeattlePublicLibrary only if database with same name does not exist
CREATE DATABASE IF NOT EXISTS SeattlePublicLibrary

In [13]:
%%sql

-- Create table CheckoutsByTitlePhysicalItemsschemafinal only if table with same name does not exist
CREATE TABLE IF NOT EXISTS SeattlePublicLibrary.checkouts_by_title_physical_items
(id STRING
,checkout_year INT
,bib_number INT
,item_barcode STRING
,item_type STRING
,collection STRING
,call_number STRING
,item_title STRING
,subjects STRING
,checkout_date_time TIMESTAMP
,load_date TIMESTAMP
)
USING PARQUET OPTIONS (path 'abfss://compose@splacceler5lmevhdeon4ym.dfs.core.windows.net/SeattlePublicLibrary/CheckoutsByTitlePhysicalItems/checkouts.parquet')

In [12]:
%%sql

--DROP TABLE SeattlePublicLibrary.Checkouts_By_Title_Physical_Items